In [1]:
import logging

import probtorch
import torch

import combinators
import foldable
import gmm
import hmm
import importance
import mcmc
import utils

logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [2]:
hmm_params = utils.vardict({
    'mu': {
        'loc': torch.arange(5, dtype=torch.float) * 2,
        'scale': torch.ones(5) * 0.25,
    },
    'sigma': {
        'loc': torch.ones(5),
        'scale': torch.ones(5) * 0.25,
    }
})
for k in range(6):
    key = 'Pi_%d' % k if k else 'Pi'
    hmm_params[key] = {'concentration': torch.ones(5)}

In [3]:
init_gmm = gmm.InitGmm(params=hmm_params, trainable=False, batch_shape=(1,))
init_hmm = hmm.InitHmm(params=hmm_params, trainable=False, batch_shape=(1,))
init_hmm = combinators.Composition(init_hmm, init_gmm)
hmm_foldable = foldable.Foldable(hmm.HmmStep(), initializer=init_hmm)
hmm_run = foldable.Reduce(hmm_foldable, generator=lambda: range(50))

In [4]:
(z_last, mu, sigma, pi), generative, _ = hmm_run()

In [5]:
data = {k: v.value for k, v in generative.variables() if 'X_' in k}

In [6]:
num_particles = 250

In [7]:
smc_hmm_params = utils.vardict({
    'mu': {
        'loc': torch.arange(5, dtype=torch.float) * 2,
        'scale': torch.ones(5),
    },
    'sigma': {
        'loc': torch.ones(5),
        'scale': torch.ones(5),
    }
})
for k in range(6):
    key = 'Pi_%d' % k if k else 'Pi'
    smc_hmm_params[key] = {'concentration': torch.ones(5)}

In [8]:
init_gmm = gmm.InitGmm(params=smc_hmm_params, trainable=True, batch_shape=(num_particles,))
init_hmm = hmm.InitHmm(params=smc_hmm_params, trainable=True, batch_shape=(num_particles,))
init_hmm = combinators.Composition(init_hmm, init_gmm)

In [9]:
hmm_step = hmm.HmmStep(batch_shape=(num_particles,))

In [10]:
rmsmc_hmm = mcmc.reduce_resample_move_smc(hmm_step, lambda: range(50), initializer=init_hmm, moves=50)

In [11]:
samples, inference, _ = rmsmc_hmm(data=data)

In [12]:
latent_states = {k: v.value for k, v in generative.variables() if 'Z_' in k}

In [13]:
inferred_latent_states = {k: v.value for k, v in inference.variables() if 'Z_' in k}

In [14]:
for t in range(50):
    t += 1
    key = 'Z_%d' % t
    accuracy = (inferred_latent_states[key] == latent_states[key]).to(dtype=torch.float).mean()
    print('RM-SMC percent accuracy at time %d: %f' % (t, accuracy * 100))

RM-SMC percent accuracy at time 1: 51.599998
RM-SMC percent accuracy at time 2: 51.599998
RM-SMC percent accuracy at time 3: 37.200001
RM-SMC percent accuracy at time 4: 66.000000
RM-SMC percent accuracy at time 5: 18.000000
RM-SMC percent accuracy at time 6: 72.799995
RM-SMC percent accuracy at time 7: 60.000004
RM-SMC percent accuracy at time 8: 72.000000
RM-SMC percent accuracy at time 9: 14.000000
RM-SMC percent accuracy at time 10: 43.599998
RM-SMC percent accuracy at time 11: 28.400002
RM-SMC percent accuracy at time 12: 27.599998
RM-SMC percent accuracy at time 13: 62.000000
RM-SMC percent accuracy at time 14: 60.799999
RM-SMC percent accuracy at time 15: 17.600000
RM-SMC percent accuracy at time 16: 77.200005
RM-SMC percent accuracy at time 17: 57.599998
RM-SMC percent accuracy at time 18: 44.000000
RM-SMC percent accuracy at time 19: 63.200005
RM-SMC percent accuracy at time 20: 45.200001
RM-SMC percent accuracy at time 21: 36.799999
RM-SMC percent accuracy at time 22: 36.3999